In [2]:
import stanza
import os
HOME = os.getcwd()
print(HOME)

from stanza.pipeline.core import DownloadMethod
NLP = stanza.Pipeline('en', download_method=DownloadMethod.REUSE_RESOURCES, use_gpu=True) # to avoid downloading the models every time
import torch

torch.cuda.is_available()


c:\Users\bouab\DEV\see-and-tell


2023-06-30 14:52:47 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-06-30 14:52:48 INFO: Using device: cuda
2023-06-30 14:52:48 INFO: Loading: tokenize
2023-06-30 14:52:59 INFO: Loading: pos
2023-06-30 14:53:00 INFO: Loading: lemma
2023-06-30 14:53:00 INFO: Loading: constituency
2023-06-30 14:53:01 INFO: Loading: depparse
2023-06-30 14:53:02 INFO: Loading: sentiment
2023-06-30 14:53:02 INFO: Loading: ner
2023-06-30 14:53:03 INFO: Done loading processors!


True

# Face recognition

In [7]:
import json 
with open('src/TBBT_embeddings_16_160_final.json', 'rb') as f:
    bbt_embeddings = json.load(f)

path_frames = os.path.join('src', 'frames')
# delete images that start with 0
fs = os.listdir(path_frames)
for f in fs:
    if not f.startswith('0'):
        os.remove(os.path.join(path_frames, f))

frames = [os.path.join(path_frames, f) for f in os.listdir(path_frames)]

In [12]:
from src.face.face_recognition import recognize_faces
for f in frames[10:20]:
    preds = recognize_faces(f, bbt_embeddings, display=False)
    print(preds)

['penny']
'NoneType' object is not iterable
[]
[]
['sheldon']
['sheldon']
['penny', 'howard']
['leonard']
['amy']
['leonard']
['amy']


In [3]:
from src.experimental.exp import get_caption
from src.face.face_recognition import recognize_faces
def get_result_image(image_path):
    # extract the caption
    caption = get_caption(image_path, use_gpu=True)
    # get the faces present in the image 
    o1 = recognize_faces(image_path, embeddings=bbt_embeddings, display=False)    
    print(caption)
    print(o1)
    return caption, o1

# Test

In [4]:
from pathlib import Path
HOME = os.getcwd()
import json
with open(os.path.join(HOME,'src','TBBT_embeddings_16_160.json')) as f:
  bbt_embeddings = json.load(f)

import os
f = os.path.join(HOME, 'src', 'frames')
frames = os.listdir(f)
frames = sorted([os.path.join(f, x) for x in frames], key=lambda x: int(os.path.basename(x)[:-4]))

In [5]:
import pickle

with open('all_captions_1.pickle', 'rb') as handle:
    captions = pickle.load(handle)


with open('all_predictions_1.pickle', 'rb') as handle:
    face_preds = pickle.load(handle)


In [6]:
import pickle 

# with open('all_predictions_1.pickle', 'wb') as handle:
#     pickle.dump(face_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('all_captions_1.pickle', 'rb') as handle:
#     captions = pickle.load(handle)


# with open('all_predictions_1.pickle', 'rb') as handle:
#     face_preds = pickle.load(handle)

In [7]:
# from src.face.face_recognition import recognize_faces
# face_pred = [recognize_faces(os.path.join(f), embeddings=bbt_embeddings) for f in frames[1:]]
assert len(face_preds) == len(captions)

In [8]:
face_preds = [[p[0] for p in ps] for ps in face_preds]

In [ ]:
import src.captions.captions_improved as ci
gcs = ci.generate_captions(captions, face_preds)

# Test refactor Face Reco

In [2]:
import os
path_frames = os.path.join('src', 'frames copy')
frames = [os.path.join(path_frames, f) for f in os.listdir(path_frames)]

dir_embs = os.path.join('src', 'emd')

os.makedirs(dir_embs, exist_ok=True)

save_emd_file = os.path.join(dir_embs, f'{len(os.listdir(dir_embs))}.json')

BBT_DIR = os.path.join('src', 'dataset', 'BigBangTheory')


import src.face_2.embeddings as e
import importlib
importlib.reload(e)
from src.face_2.embeddings import build_classes_embeddings

bbt = build_classes_embeddings(directory=BBT_DIR)